In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
import joblib
import os

# Configuración para visualización
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_palette("viridis")
os.makedirs('results_initial_hyper', exist_ok=True)

# Función para cargar y preparar los datos
def load_data(file_path):
    df = pd.read_csv(file_path)
    
    # Verificar y gestionar valores faltantes
    print(f"Valores faltantes en el dataset: {df.isnull().sum().sum()}")
    
    # Identificar columnas categóricas automáticamente (si es necesario)
    categorical_features = []
    for col in df.columns:
        if df[col].dtype == 'object' or col in ['tipo_edificacion', 'calidade_materiais', 
                                               'cor_favorita_propietario', 'acceso_transporte_publico',
                                               'orientacion', 'eficiencia_enerxetica']:
            categorical_features.append(col)
    
    print(f"Características categóricas detectadas: {categorical_features}")
    
    # Separamos features y target
    X = df.drop(['prezo_euros', 'id'], axis=1, errors='ignore')
    y = df['prezo_euros']
    
    print(f"Forma del dataset: {df.shape}")
    print(f"Features incluidas: {X.columns.tolist()}")
    
    return X, y, categorical_features

# FASE 1: Entrenamiento base y análisis
def train_initial_model(X, y, cat_features=None, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.1, random_state=random_state
    )
    
    initial_params = {
        'iterations': 10000,
        'learning_rate': 0.05,
        'depth': 3,
        'loss_function': 'MAE',
        'eval_metric': 'MAE',
        'random_seed': random_state,
        'verbose': 200
    }
    
    print("\n=== Entrenando modelo inicial para análisis ===")
    
    # Crear pool de datos para CatBoost
    train_pool = Pool(X_train, y_train, cat_features=cat_features)
    test_pool = Pool(X_test, y_test, cat_features=cat_features)
    
    # Entrenar modelo
    model = CatBoostRegressor(**initial_params)
    model.fit(train_pool, eval_set=test_pool, use_best_model=True)
    
    # Evaluar modelo
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    print(f"Métricas del modelo inicial:")
    print(f"  MAE: {mae:.2f}")
    print(f"  RMSE: {rmse:.2f}")
    print(f"  R2: {r2:.4f}")
    
    
    
    # Analizar importancia de características
    feature_importance = model.get_feature_importance()
    feature_names = X.columns
    importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})
    importance_df = importance_df.sort_values('Importance', ascending=False)
    """
    plt.figure(figsize=(12, 8))
    sns.barplot(x='Importance', y='Feature', data=importance_df.head(20))
    plt.title('Top 20 características más importantes', fontsize=15)
    plt.tight_layout()
    plt.savefig('results_initial_hyper/initial_feature_importance.png')
    
    """
    
    
    print("\nAnálisis de características completado - Ver 'results_initial_hyper/initial_feature_importance.png'")
    
    # Devolver todo lo necesario para optimización
    return model, X_train, X_test, y_train, y_test, cat_features, importance_df

# FASE 2: Optimización con Grid Search (solo sobre los parámetros iniciales)
def optimize_with_grid_search(X_train, y_train, X_test, y_test, cat_features=None, random_state=42):
    print("\n=== Optimización con Grid Search ===")
    
    # Grid de parámetros a evaluar (solo los parámetros iniciales)
    param_grid = {
        'iterations': [5000],
        'learning_rate': [0.1,0.08, 0.05, 0.03, 0.01],
        'depth': [1,2,3,4,5,6,7],
    }
    
    # Crear pool para entrenamiento
    train_pool = Pool(X_train, y_train, cat_features=cat_features)
    
    # Configurar modelo base para GridSearch
    base_model = CatBoostRegressor(
        loss_function='MAE',
        eval_metric='MAE',
        random_seed=random_state,
        verbose=100
    )
    
    # Configurar GridSearchCV
    grid_search = GridSearchCV(
        estimator=base_model,
        param_grid=param_grid,
        cv=3,  # Usar validación cruzada con 3 folds
        scoring='neg_mean_absolute_error',
        n_jobs=-1,  # Usar todos los núcleos disponibles
        verbose=1
    )
    
    # Entrenar Grid Search
    start_time = time.time()
    grid_search.fit(X_train, y_train, cat_features=cat_features)
    gs_time = time.time() - start_time
    
    # Mejor valor de MAE encontrado
    print(f"Tiempo de Grid Search: {gs_time:.2f} segundos")
    print(f"Mejor MAE encontrado: {-grid_search.best_score_:.2f}")
    print(f"Mejores parámetros: {grid_search.best_params_}")
    
    # Evaluar mejor modelo en conjunto de prueba
    best_gs_model = grid_search.best_estimator_
    y_pred = best_gs_model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    
    print(f"MAE en conjunto de prueba: {mae:.2f}")
    
    # Guardar modelo optimizado en formato CatBoost .cbm
    best_gs_model.save_model('results_initial_hyper/best_model_long.cbm')
    print("\nModelo guardado como 'results_initial_hyper/best_model.cbm'")
    
    # Guardar resultados
    gs_results = pd.DataFrame(grid_search.cv_results_)
    gs_results.to_csv('results_initial_hyper/grid_search_results.csv', index=False)
    
    return best_gs_model, grid_search.best_params_

# Fase de ejecución del script
if __name__ == "__main__":
    # Cargar datos
    X, y, cat_features = load_data('train_processed.csv')
    
    # Entrenar el modelo inicial
    model, X_train, X_test, y_train, y_test, cat_features, importance_df = train_initial_model(X, y, cat_features)
    
    # Guardar modelo optimizado en formato CatBoost .cbm
    model.save_model('results_initial_hyper/best_model_initial.cbm')
    print("\nModelo guardado como 'results_initial_hyper/best_model.cbm'")

    # Evaluar mejor modelo en conjunto de prueba
    
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    
    print(f"MAE en conjunto de prueba: {mae:.2f}")
    
    # Guardar resultados
    #gs_results = pd.DataFrame(model.cv_results_)
    #gs_results.to_csv('results_initial_hyper/grid_search_results.csv', index=False)
    
    # Optimización de hiperparámetros con GridSearch
    #best_model, best_params = optimize_with_grid_search(X_train, y_train, X_test, y_test, cat_features)


Valores faltantes en el dataset: 0
Características categóricas detectadas: []
Forma del dataset: (20000, 47)
Features incluidas: ['superficie_interior_m2', 'superficie_exterior_m2', 'numero_habitacions', 'numero_banos', 'ano_construccion', 'lonxitude', 'latitude', 'temperatura_media_mes_construccion', 'distancia_centro_km', 'distancia_escola_km', 'indice_criminalidade', 'numero_arboles_xardin', 'edad_vivienda', 'superficie_por_habitacion', 'superficie_total', 'ratio_interior_exterior', 'densidad_banos', 'densidad_habitaciones', 'dist_coruna', 'dist_vigo', 'dist_santiago', 'calidad_edad', 'banos_por_habitacion', 'orientacion_valor', 'eficiencia_valor', 'calidade_valor', 'transporte_valor', 'tipo_Apartamento', 'tipo_Casa', 'tipo_Chalet adosado', 'color_Amarelo', 'color_Azul', 'color_Branco', 'color_Negro', 'color_Verde', 'color_Vermello', 'tipo_Apartamento.1', 'tipo_Casa.1', 'tipo_Chalet adosado.1', 'color_Amarelo.1', 'color_Azul.1', 'color_Branco.1', 'color_Negro.1', 'color_Verde.1', 'c

In [6]:

# Verificar la existencia del modelo entrenado
model_path = 'results_initial_hyper/best_model_initial.cbm'
if not os.path.exists(model_path):
    print(f"¡Error! No se encuentra el modelo en {model_path}")
    print("Por favor, asegúrate de ejecutar primero el script de entrenamiento")
    exit(1)

print("=== Generando predicciones para el conjunto de prueba ===")

# Cargar el dataset de prueba
print("Cargando datos de prueba...")
test_df = pd.read_csv('test_processed.csv')

# Extraer los IDs para el submission
ids = test_df['id']

# Verificar características categóricas (usando las mismas que en entrenamiento)
categorical_features = []
for col in test_df.columns:
    if test_df[col].dtype == 'object' or col in ['tipo_edificacion', 'calidade_materiais', 
                                              'cor_favorita_propietario', 'acceso_transporte_publico',
                                              'orientacion', 'eficiencia_enerxetica']:
        categorical_features.append(col)

print(f"Características categóricas detectadas: {categorical_features}")

# Preparar los datos para la predicción (eliminar cualquier columna que no sea una feature)
X_test = test_df.drop(['id', 'Unnamed: 0'], axis=1, errors='ignore')
if 'prezo_euros' in X_test.columns:
    X_test = X_test.drop(['prezo_euros'], axis=1, errors='ignore')

print(f"Forma del dataset de prueba: {X_test.shape}")

# Cargar el modelo entrenado
print("Cargando el modelo entrenado...")
model = CatBoostRegressor()
model.load_model(model_path)

# Realizar predicciones
print("Generando predicciones...")
predictions = model.predict(X_test)

# Crear el dataframe de submission
submission = pd.DataFrame({
    'id': ids,
    'prezo_euros': predictions
})

# Guardar el archivo de submission
submission_path = 'submission_catboost_initial_hyper_initial_simple.csv'
submission.to_csv(submission_path, index=False)

print(f"¡Completado! Archivo de submission generado en {submission_path}")
print(f"Primeras filas del archivo de submission:")
print(submission.head(10))

# Estadísticas de las predicciones
print("\nEstadísticas de las predicciones:")
print(f"Mínimo: {predictions.min():.2f}")
print(f"Máximo: {predictions.max():.2f}")
print(f"Media: {predictions.mean():.2f}")
print(f"Mediana: {np.median(predictions):.2f}")
print(f"Desviación estándar: {predictions.std():.2f}")

=== Generando predicciones para el conjunto de prueba ===
Cargando datos de prueba...
Características categóricas detectadas: []
Forma del dataset de prueba: (10000, 45)
Cargando el modelo entrenado...
Generando predicciones...
¡Completado! Archivo de submission generado en submission_catboost_initial_hyper_long.csv
Primeras filas del archivo de submission:
      id    prezo_euros
0   2309  465389.731641
1  22405  147166.867508
2  23398  422953.382795
3  25059  262508.070719
4   2665  586974.908665
5   8512   78392.320081
6   5149  599309.750207
7   7791  274498.132714
8  11312  109116.843607
9  19044  327702.003566

Estadísticas de las predicciones:
Mínimo: 43170.38
Máximo: 829299.20
Media: 222292.58
Mediana: 151767.43
Desviación estándar: 163590.92
